In [1]:
import secretflow as sf

# In case you have a running secretflow runtime already.
sf.shutdown()
# 基于这四个party，我们将会建立3个设备。
# 一个基于 alice 的PYU设备
# 一个基于 dave 的PYU设备
# 一个基于 alice , bob 和 carol 的SPU设备
sf.init(['alice', 'bob', 'carol', 'dave'], address='local')

2023-06-01 20:52:26,437	INFO worker.py:1538 -- Started a local Ray instance.


In [3]:
# sf.utils.testing.cluster_def 是一个helper通过寻找未占用的端口来创建一个设置。
aby3_config = sf.utils.testing.cluster_def(parties=['alice', 'bob', 'carol'])

aby3_config

{'nodes': [{'party': 'alice', 'address': '127.0.0.1:54587'},
  {'party': 'bob', 'address': '127.0.0.1:60045'},
  {'party': 'carol', 'address': '127.0.0.1:46017'}],
 'runtime_config': {'protocol': 3, 'field': 3}}

In [4]:
# 随后我们用 aby3_config 来创建一个SPU设备并检查其 cluster_def 。
spu_device = sf.SPU(aby3_config)

spu_device.cluster_def

{'nodes': [{'party': 'alice', 'address': '127.0.0.1:54587'},
  {'party': 'bob', 'address': '127.0.0.1:60045'},
  {'party': 'carol', 'address': '127.0.0.1:46017'}],
 'runtime_config': {'protocol': 3, 'field': 3}}

In [5]:
# 们创建两个PYU设备。
alice, dave = sf.PYU('alice'), sf.PYU('dave')

In [6]:
bank_account = [{'id': 12345, 'deposit': 1000.25}, {'id': 12345, 'deposit': 100000.25}]

# 这个东西他先to给了alice然后再to给了spu单元，就是让alice做pyu的意思？ 不是 傻逼了 前面定义了
temp = sf.to(alice,bank_account)
bank_account_spu = sf.to(alice, bank_account).to(spu_device)

(_run pid=27862) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=27862) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=27862) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
(_run pid=27862) INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
(_run pid=27862) WARNING:jax._src.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [7]:
# 首先，我们用一个PYU设备创建一个PYU object。
def debit_amount():
    return 10


debit_amount_pyu = alice(debit_amount)()
debit_amount_pyu

In [8]:
# 然后我们将debit_amount_pyu从PYU传到SPU，我们将会得到一个SPU object作为结果。
debit_amount_spu = debit_amount_pyu.to(spu_device)

debit_amount_spu

In [9]:
def deduce_from_account(bank_account, amount):
    new_bank_account = []

    for account in bank_account:
        account['deposit'] = account['deposit'] - amount
        new_bank_account.append(account)

    return new_bank_account


new_bank_account_spu = spu_device(deduce_from_account)(
    bank_account_spu, debit_amount_spu
)

new_bank_account_spu

# newbankaccountspu 也是一个 SPU Object 。 但它与 bank_account_spu 和 debit_amount_spu 有点不同！

# bank_account_spu 位于host，因此您可以直接从host检查值。
# debit_amount_spu 位于 alice，因此只有 alice 可以检查值。
# newbankaccount_spu 位于spu，spu的每一方都有一份shares。 如果没有 sf.reveal ，您将无法直接检查该值。
# 这里的host是啥意思啊

(raylet) [2023-06-01 20:53:26,373 E 27755 27755] (raylet) node_manager.cc:3097: 3 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 037df0d88cd8839084e3c3d6f6ec17b656e2613e2b54b4af9bde5941, IP: 10.136.75.128) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 10.136.75.128`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


In [10]:
new_bank_account_pyu = new_bank_account_spu.to(dave)

new_bank_account_pyu

In [12]:
sf.reveal(new_bank_account_pyu)

[{'deposit': array(990.25, dtype=float32), 'id': array(12345, dtype=int32)},
 {'deposit': array(99990.25, dtype=float32), 'id': array(12345, dtype=int32)}]

In [13]:
sf.reveal(new_bank_account_spu)

[{'deposit': array(990.25, dtype=float32), 'id': array(12345, dtype=int32)},
 {'deposit': array(99990.25, dtype=float32), 'id': array(12345, dtype=int32)}]

(raylet) [2023-06-01 21:00:26,382 E 27755 27755] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 037df0d88cd8839084e3c3d6f6ec17b656e2613e2b54b4af9bde5941, IP: 10.136.75.128) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 10.136.75.128`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
